##### RAG Pipeline (Data Ingestion -> Chunking -> Embedding -> Vector DB)

In [2]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path